In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import io
import os
import re
import string
import shutil
from tqdm.notebook  import tqdm
import pickle
from IPython.display import clear_output
from datetime import datetime

import tensorflow as tf
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Lambda, BatchNormalization
from tensorflow.keras.layers import TextVectorization

import keras.backend as K
from keras.models import Sequential
from keras.preprocessing import text, sequence
from keras.utils import np_utils

gpu = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)


%load_ext tensorboard
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

In [40]:
def clean(doc):
    r_string = set(doc) - set(['A', 'B', 'C', 'D', 'E', 'F', 'G', 
                                    'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 
                                    'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
                                    'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 
                                    'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 
                                    's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ',
                                      '1','2','3','4','5','6','7','8','9', '0','Á', 'Å', 'Æ', 'É',
                                  'Í', 'Î', 'Ö', '×', 'Ø', 'Ú', 'Ü', 'Þ', 'à', 'á', 'â', 'ã', 'ä', 'å', 
                                  'ç', 'è', 'é', 'ê', 'ë', 'ì', 'í', 'î', 'ñ', 'ò', 'ó', 'ô', 'ö', 'ø', 'ú',
                                  'û', 'ü', 'Ā', 'ā', 'ă', 'ć', 'č', 'Đ', 'đ', 'ė', 'ī', 'Ł', 'ł', 'ń', 'Ō', 'ō',
                                  'ś', 'ş', 'š', 'ū', 'ų', 'Ż',
                                  'ž', 'ơ', 'ư',
                                  'ḥ', 'ṃ', 'ṅ', 'ṣ', 'ṭ', 'ṯ', 'ả', 'ấ', 'ầ', 'ắ', 'ễ', 'ệ', 'ị', 'ớ', 'ử', 'ỳ','½','.',
                            
                                 ])
    r_list = sorted(list(r_string))
    print('Removed  --' + ''.join(r_list) +  '\n')
    document_ = doc

    for r in r_list:
        document_ =  document_.replace(r,"")

    return document_


In [50]:
corpus = tf.io.read_file('./wiki.train.tokens')
corpus =tf.strings.lower(corpus)
corpus = corpus.numpy()
corpus = corpus.decode('utf-8')
corpus = clean(corpus)
one_doc = [corpus]
corpus =  corpus.split('.')
corpus = [line for line in corpus if line is not ' ']

Removed  --
!"#$%&'()*+,-/:;<=>?@[\]^`|~¡£¥§°±²³µ·ʻʿ̃αβγκμСавекостяاحصلنه्กงณตมยรลัาิ่์გდვზიკორსუცძწხჯ჻‑–—‘’“”„†…′″⁄₤€₹⅓⅔→−≤☉♭♯〈〉のァアキスットプュリルヴ・動場大戦攻機殻火礮空隊﻿～



In [59]:
print(len(one_doc[0]))
one_doc[0][:100]


10347718


'   valkyria chronicles iii    senjō no valkyria 3  unk chronicles  japanese  3  lit . valkyria of th'

In [4]:
for i in range(10):
    print(corpus[i] , '\n')

   valkyria chronicles iii    senjō no valkyria 3  unk chronicles  japanese  3  lit  

 valkyria of the battlefield 3   commonly referred to as valkyria chronicles iii outside japan  is a tactical role  playing video game developed by sega and media 

vision for the playstation portable  

 released in january 2011 in japan  it is the third game in the valkyria series  

 unk the same fusion of tactical and real  time gameplay as its predecessors  the story runs parallel to the first game and follows the  nameless   a penal military unit serving the nation of gallia during the second europan war who perform secret black operations and are pitted against the imperial unit  unk raven   

  the game began development in 2010  carrying over a large portion of the work done on valkyria chronicles ii  

 while it retained the standard features of the series  it also underwent multiple adjustments  such as making the game more unk for series newcomers  

 character designer unk honjou and com

In [5]:
path = "./corpus" +'_file.pkl'
with open(path, 'wb') as files:
    pickle.dump(corpus, files)

In [4]:
path = "./corpus" +'_file.pkl'
with open(path, 'rb') as files:
     corpus = pickle.load(files)
        
max_document_length =16

max_document_length = 10347718

tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(corpus)
word2id = tokenizer.word_index
word2id['PAD'] = 0

id2word = {v:k for k, v in word2id.items()}
wids = [[ word2id[w] for w in text.text_to_word_sequence(doc)] 
        for doc in corpus]


# wids = sequence.pad_sequences(wids, maxlen=max_document_length)

In [5]:
(len(wids[0]))
print(wids[0][0:10])

[3768, 3831, 859, 18089, 75, 3768, 80, 3, 3831, 759]


In [6]:
vocab_size = len(word2id)
embedding_dim = 512

print('Vocabulary Size:', vocab_size)
print('Vocabulary Sample:', list(word2id.items())[:10])

Vocabulary Size: 28673
Vocabulary Sample: [('the', 1), ('of', 2), ('unk', 3), ('and', 4), ('in', 5), ('to', 6), ('a', 7), ('was', 8), ('on', 9), ('as', 10)]


In [7]:
window_size=2
    
    # def generate_context_word_pairs(corpus=wids, window_size=3, vocab_size=vocab_size):
#     context_length = window_size*2 -1   
#     for doc in corpus:
 
#         sentence_length = doc.shape[0]
#         # print("sentence_length" , sentence_length)
#         context_words = []
#         label_words = []
#         # print(doc)

#         for index, word in enumerate(doc):
#             # print("index" , index)
            
#             start = index - window_size
#             end = index + window_size
            
#             # print("start" , start)
#             # print("end" , end)
            
#             input_ = []
#             for i in range(start, end) :
#                 if (0 <= i and i< sentence_length ):
#                     # print(i)
#                     if(i != index):
#                         input_.append(doc[i])


#             context_words.append(input_)
#             label_words.append(word)

#         # print(context_words)
#         x = sequence.pad_sequences(context_words, maxlen=context_length)
#         y = np_utils.to_categorical(label_words, vocab_size)

#         yield x, y

In [8]:

batch_size = 1024

def generate_context_word_pairs(corpus=wids, window_size=3, vocab_size=vocab_size):
    
    context_length = window_size*2 -1

   
    for doc in corpus:
 
        sentence_length = len(doc)
        # print("sentence_length" , sentence_length)
        context_words = []
        label_words = []
        # print(doc)
        batch_i = 0

        for index, word in enumerate(doc):
#             print("index" , index)
#             print("word" , index)
            
            start = index - window_size
            end = index + window_size
            
            # print("start" , start)
            # print("end" , end)
            
            input_ = []
            for i in range(start, end) :
                if (0 <= i and i< sentence_length ):
                    # print('word', doc[i])
                    
                    if(i != index):
                        input_.append(doc[i])
                        
            # print(input_)
                        
            context_words.append(input_)
            label_words.append(word)
            
            batch_i += 1
            if(batch_i == batch_size):
                x = sequence.pad_sequences(context_words, maxlen=context_length)
                y = np_utils.to_categorical(label_words, vocab_size)
                batch_i = 0
                context_words = []
                label_words = [] 
                
                yield x, y
            
        
        
# Test this out for some samples
i = 0
for x, y in generate_context_word_pairs(corpus=wids, window_size=window_size, vocab_size=vocab_size):
        print(x, y)
        print(len(x))
        print(len(y[0]))

        # print('Context (X):', [id2word[w] for w in x[5]], '-> Target (Y):', y[5])
        
        if i == 2:
            break
        i += 1


In [11]:
# build CBOW architecture
cbow = Sequential()
cbow.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length= window_size*2, name = 'w2v_embedding'))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embedding_dim,)))
cbow.add(tf.keras.layers.BatchNormalization())
cbow.add(Dense(vocab_size, activation='softmax'))


acc = tf.keras.metrics.CategoricalAccuracy('accuracy')
loss = tf.keras.losses.CategoricalCrossentropy()
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
cbow.compile(loss='categorical_crossentropy', optimizer=opt, metrics=[acc])

# view model summary
print(cbow.summary())

logs = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logs + "/metrics")




Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
w2v_embedding (Embedding)    (None, 4, 512)            14680576  
_________________________________________________________________
lambda_2 (Lambda)            (None, 512)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
_________________________________________________________________
dense_2 (Dense)              (None, 28673)             14709249  
Total params: 29,391,873
Trainable params: 29,390,849
Non-trainable params: 1,024
_________________________________________________________________
None


In [139]:
epochs = 10
epoch = 0
losses= []
accuracies = []
step = 0
win_size = 2

In [170]:
# tensorboard --logdir logs
epoch = 0

In [ ]:

while epoch < epochs:
    print("\nStart of epoch %d" % (epoch,))
    epoch += 1
   
    for x, y in generate_context_word_pairs(corpus=wids, window_size=win_size, vocab_size=vocab_size):

        step = step + 1
        with tf.GradientTape() as tape:
   
            y_ = cbow(x) 
            loss_value = loss(y, y_)
        
            grads = tape.gradient(loss_value, cbow.trainable_weights)    
            opt.apply_gradients(zip(grads, cbow.trainable_weights))
        
        acc.update_state(y, y_)
        train_acc = acc.result()

        losses.append(loss_value)
        accuracies.append(train_acc )
       
        print('Loss {}   Accuracy {}'.format(loss_value ,train_acc*100))
        clear_output(wait=True)
        
        with file_writer.as_default():
            tf.summary.scalar('loss', loss_value, step =step)
            tf.summary.scalar('accuracy', train_acc, step = step)
            


In [172]:
cbow.save('.\cbow')
np.save('Loss',losses)
np.save('Accuracy',accuracies)

INFO:tensorflow:Assets written to: .\cbow\assets
